In [ ]:
from PIL import Image
import os
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers,callbacks
from keras.layers import (Input, Dense, Activation, ZeroPadding2D,
BatchNormalization, Flatten, Conv2D, concatenate, Lambda)

from keras.layers import (AveragePooling2D, MaxPooling2D, Dropout,
GlobalMaxPooling2D, GlobalAveragePooling2D)

from keras.models import Model, load_model
from keras import regularizers, optimizers

from sklearn.model_selection import train_test_split



In [ ]:
early_stopping=callbacks.EarlyStopping( 
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,)

model=keras.Sequential([layers.Dense(units=512),
                        layers.Dense(units=512,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dropout(0.3),
                        layers.Flatten(),
                        layers.Dense(units=256,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dropout(0.3),
                        layers.Dense(128,activation='relu'),
                        layers.Dense(1),
                        
                       ])

In [ ]:
#dataset=image_dataset_from_directory('/kaggle/input/self-driving-cars-dataset/dataset/dataset/')
image_folder='../input/autonomouscarsdataset/dataset/IMG'
data_csv=pd.read_csv('../input/autonomouscarsdataset/dataset/driving_log.csv')

center = data_csv[data_csv.columns[0]].values
left = data_csv[data_csv.columns[1]].values
right = data_csv[data_csv.columns[2]].values
steering = data_csv[data_csv.columns[3]].values

data_csv.head()
print(center)

In [ ]:
total=len(steering)
print(total)

In [ ]:
def random_flip(img,st_angle):
    image = cv2.flip(img, 1)
    st_angle = -st_angle
    
    return image, st_angle

In [ ]:
X_train=[]
y_train=[]
steering_factor=0.2
for i in range(total):
    for j in range(3):
        if j==0:
            img = cv2.imread(os.path.join(image_folder,center[i].split('IMG\\')[1]))
            steering_angle = steering[i]
        if j==1:
            img=cv2.imread(os.path.join(image_folder,left[i].split('IMG\\')[1]))
            steering_angle=steering[i]+steering_factor
        if j==2:
            img=cv2.imread(os.path.join(image_folder,right[i].split('IMG\\')[1]))
            steering_angle=steering[i]-steering_factor
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,1]
    img = cv2.resize(img,(100,100))
    X_train.append(img)
    y_train.append(steering_angle)
    flip_img,st_angle=random_flip(img,steering_angle)
    X_train.append(flip_img)
    y_train.append(st_angle)
    

X_train = np.array(X_train)
X_train= np.reshape(X_train,[X_train.shape[0],X_train.shape[1],X_train.shape[2],1])

y_train= np.array(y_train)
y_train = np.reshape(y_train,[y_train.shape[0],1])
    
print(X_train.shape)
print(y_train.shape)
         

In [ ]:
x_train,x_test,Y_train,Y_test = train_test_split(X_train,y_train,random_state=42,test_size=.20)

In [ ]:
def model(height,width):
        
    x_input = Input(shape=(height,width,1))
    
    x = Lambda(lambda x: x/127.5-1.0)(x_input)
    
    x = Conv2D(32,(3,3),activation='relu',padding='same')(x_input)
    
    x = Conv2D(32,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
    
    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Flatten()(x)
    x = Dropout(0.5)(x)
       
    x = BatchNormalization()(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(64)(x)
    x = Dense(1)(x)
    
    model = Model(inputs=x_input,outputs=x,name='model')
    
    return model
    
model = model(100,100)
print(model.summary())


model.compile(loss='mse',
             optimizer='adam',
             metrics=['mse'])

hist = model.fit(x_train,Y_train,validation_data=(x_test,Y_test),batch_size=32,epochs=10)

In [ ]:
print(min(hist.history['loss']))

In [ ]:
model.save('Saved_model.h5')